In [1]:
import numpy as np
import pandas as pd 

from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.inference import VariableElimination
from pgmpy.estimators import MaximumLikelihoodEstimator

c:\Users\Bruger\anaconda3\envs\python_3_10_16\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
#!pip install pgmpy # https://pgmpy.org/detailed_notebooks/10.%20Learning%20Bayesian%20Networks%20from%20Data.html
# https://pgmpy.org/metrics/metrics.html

In [2]:
df = pd.read_csv("transport.csv")
df = df.drop(columns="Unnamed: 0")
df

,Age,Income,Education,Residency,Transport
0,middle,low,high,big,train
1,young,low,low,big,car
2,middle,low,low,small,car
3,old,low,high,big,car
4,middle,medium,high,small,car
...,...,...,...,...,...
995,middle,medium,low,small,car
996,middle,high,high,big,car
997,middle,medium,high,small,car
998,middle,medium,high,big,car


In [3]:
from pgmpy.factors.discrete.CPD import TabularCPD

def print_full(cpd):
    backup = TabularCPD._truncate_strtable
    TabularCPD._truncate_strtable = lambda self, x: x
    print(cpd)
    TabularCPD._truncate_strtable = backup

In [ ]:
def do_bayesian_network(network):
    model = BayesianNetwork(network)
    model.fit(df, estimator=MaximumLikelihoodEstimator)

    # Check the learned CPDs
    print(model.get_cpds())
    
    model.check_model()
    print_full(model)
    print_mle(model)

    return model

def print_mle(model):
    mle = MaximumLikelihoodEstimator(model, df)

    print(mle.estimate_cpd("Age"))  # unconditional
    print(mle.estimate_cpd("Residency"))  # conditional
    print(mle.estimate_cpd("Education"))  # conditional
    print(mle.estimate_cpd("Income"))  # conditional
    print(mle.estimate_cpd("Transport"))  # conditional

In [69]:
do_bayesian_network([("Age", "Income"), ("Age", "Education"), ("Income", "Transport"), ("Income", "Residency")])

[<TabularCPD representing P(Age:3) at 0x19f6de65240>, <TabularCPD representing P(Income:3 | Age:3) at 0x19f6de666b0>, <TabularCPD representing P(Education:2 | Age:3) at 0x19f6de64bb0>, <TabularCPD representing P(Transport:2 | Income:3) at 0x19f6de66b90>, <TabularCPD representing P(Residency:2 | Income:3) at 0x19f6e01c790>]
BayesianNetwork with 5 nodes and 4 edges
+-------------+-------+
| Age(middle) | 0.538 |
+-------------+-------+
| Age(old)    | 0.2   |
+-------------+-------+
| Age(young)  | 0.262 |
+-------------+-------+
+------------------+-----+---------------------+
| Income           | ... | Income(medium)      |
+------------------+-----+---------------------+
| Residency(big)   | ... | 0.5674740484429066  |
+------------------+-----+---------------------+
| Residency(small) | ... | 0.43252595155709345 |
+------------------+-----+---------------------+
+-----------------+---------------------+----------+---------------------+
| Age             | Age(middle)         | Age(ol

In [5]:
do_bayesian_network([("Age", "Education"), ("Education", "Income"), ("Income", "Residency"), ("Residency", "Transport")])

[<TabularCPD representing P(Age:3) at 0x1f4d858e8c0>, <TabularCPD representing P(Education:2 | Age:3) at 0x1f4d858ffd0>, <TabularCPD representing P(Income:3 | Education:2) at 0x1f4d8600a00>, <TabularCPD representing P(Residency:2 | Income:3) at 0x1f4d8600670>, <TabularCPD representing P(Transport:2 | Residency:2) at 0x1f4d8601ae0>]
BayesianNetwork with 5 nodes and 4 edges
+-------------+-------+
| Age(middle) | 0.538 |
+-------------+-------+
| Age(old)    | 0.2   |
+-------------+-------+
| Age(young)  | 0.262 |
+-------------+-------+
+------------------+-----+---------------------+
| Income           | ... | Income(medium)      |
+------------------+-----+---------------------+
| Residency(big)   | ... | 0.5674740484429066  |
+------------------+-----+---------------------+
| Residency(small) | ... | 0.43252595155709345 |
+------------------+-----+---------------------+
+-----------------+---------------------+----------+---------------------+
| Age             | Age(middle)        

In [6]:
do_bayesian_network([("Age", "Education"), ("Education", "Income"), ("Income", "Transport"), ("Residency", "Transport")])

[<TabularCPD representing P(Age:3) at 0x1f4d8601cf0>, <TabularCPD representing P(Education:2 | Age:3) at 0x1f4d8601c30>, <TabularCPD representing P(Income:3 | Education:2) at 0x1f4d8600760>, <TabularCPD representing P(Transport:2 | Income:3, Residency:2) at 0x1f4d8602230>, <TabularCPD representing P(Residency:2) at 0x1f4d8602500>]
BayesianNetwork with 5 nodes and 4 edges
+-------------+-------+
| Age(middle) | 0.538 |
+-------------+-------+
| Age(old)    | 0.2   |
+-------------+-------+
| Age(young)  | 0.262 |
+-------------+-------+
+------------------+-------+
| Residency(big)   | 0.638 |
+------------------+-------+
| Residency(small) | 0.362 |
+------------------+-------+
+-----------------+---------------------+----------+---------------------+
| Age             | Age(middle)         | Age(old) | Age(young)          |
+-----------------+---------------------+----------+---------------------+
| Education(high) | 0.31970260223048325 | 0.345    | 0.31679389312977096 |
+------------

In [8]:
do_bayesian_network([("Age", "Education"), ("Age", "Income"), ("Education", "Income"), ("Income", "Residency"), ("Residency", "Transport"), ("Income", "Residency")])

[<TabularCPD representing P(Age:3) at 0x1f4d858fbb0>, <TabularCPD representing P(Education:2 | Age:3) at 0x1f4d858fac0>, <TabularCPD representing P(Income:3 | Age:3, Education:2) at 0x1f4d8603f10>, <TabularCPD representing P(Residency:2 | Income:3) at 0x1f4d8603880>, <TabularCPD representing P(Transport:2 | Residency:2) at 0x1f4d8602fe0>]
BayesianNetwork with 5 nodes and 5 edges
+-------------+-------+
| Age(middle) | 0.538 |
+-------------+-------+
| Age(old)    | 0.2   |
+-------------+-------+
| Age(young)  | 0.262 |
+-------------+-------+
+------------------+-----+---------------------+
| Income           | ... | Income(medium)      |
+------------------+-----+---------------------+
| Residency(big)   | ... | 0.5674740484429066  |
+------------------+-----+---------------------+
| Residency(small) | ... | 0.43252595155709345 |
+------------------+-----+---------------------+
+-----------------+---------------------+----------+---------------------+
| Age             | Age(middle) 

In [ ]:
# calculate how good it is